In [1]:
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import seaborn as sns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


df = pd.read_excel('smFISH_screen_annotations.xlsx', header=[2])
df = df.iloc[1:201, 1:11]

In [2]:
df_VNC_1 = pd.read_csv('scoringResults/VNC/rater1.csv')
df_VNC_2 = pd.read_csv('scoringResults/VNC/rater2.csv')
df_VNC_3 = pd.read_csv('scoringResults/VNC/rater3.csv')

df_NMJ_1 = pd.read_csv('scoringResults/NMJ/rater1.csv')
df_NMJ_2 = pd.read_csv('scoringResults/NMJ/rater2.csv')
df_NMJ_3 = pd.read_csv('scoringResults/NMJ/rater3.csv')

df_CB_1 = pd.read_csv('scoringResults/CB/rater1.csv')
df_CB_2 = pd.read_csv('scoringResults/CB/rater2.csv')
df_CB_3 = pd.read_csv('scoringResults/CB/rater3.csv')

df_MB_1 = pd.read_csv('scoringResults/MB/rater1.csv')
df_MB_2 = pd.read_csv('scoringResults/MB/rater2.csv')
df_MB_3 = pd.read_csv('scoringResults/MB/rater3.csv')

df_OL_1 = pd.read_csv('scoringResults/OL/rater1.csv')
df_OL_2 = pd.read_csv('scoringResults/OL/rater2.csv')
df_OL_3 = pd.read_csv('scoringResults/OL/rater3.csv')

df_nerve_1 = pd.read_csv('scoringResults/nerve/rater1.csv')
df_nerve_2 = pd.read_csv('scoringResults/nerve/rater2.csv')
df_nerve_3 = pd.read_csv('scoringResults/nerve/rater3.csv')

In [3]:
def conflict_resolver(df1, df2, df3, compartment, columns_drop):
    merge_column = compartment + '_id'
    df1 = df1.merge(df[merge_column], left_on='figure_id', right_on = merge_column, how='inner')
    df2 = df2.merge(df[merge_column], left_on='figure_id', right_on = merge_column, how='inner')
    df3 = df3.merge(df[merge_column], left_on='figure_id', right_on = merge_column, how='inner')
    
    df1 = df1.drop(columns= merge_column, axis=1)
    df2 = df2.drop(columns= merge_column, axis=1)
    df3 = df3.drop(columns= merge_column, axis=1)
    
    df_common = df1.merge(df2, on='figure_id', how='inner')
    common_index = list(df_common.merge(df3, on='figure_id', how='inner')['figure_id'])
    common_columns = list((df2.columns.intersection(df1.columns)).intersection(df3.columns))
    
    common_changed = []
    original_dfs = []
    for dataframe in [df1, df2, df3]:
        dfcommon = dataframe[common_columns].set_index('figure_id')
        dfcommon = dfcommon[dfcommon.index.isin(common_index)].sort_index()
        dfcommon = dfcommon.drop(columns= columns_drop)
        dfcommon_changed = dfcommon.replace({'None': 0, 'Low': 1/3, 'Moderate': 2/3, 'High': 1, 'No': 0, 'Yes': 1})
        dfcommon.fillna('Non applicable / Missing Image', inplace=True)
        dfcommon_changed.fillna('Non applicable / Missing Image', inplace=True)
        common_changed.append(dfcommon_changed)
        original_dfs.append(dfcommon)
    
    df_1, df_2, df_3 = common_changed[0], common_changed[1], common_changed[2]
    df1orig, df2orig, df3orig = original_dfs[0], original_dfs[1], original_dfs[2]
    
    df_1.fillna('Unresolved', inplace=True)
    
    agreement_3_list = []
    agreement_1_2_list = []
    agreement_1_3_list = []
    agreement_2_3_list = []

    dffinal = pd.DataFrame(index=df_1.index, columns=df_1.columns)

    for column in df_1.columns:

        agreement_3_list = ((df_1[column] == df_2[column]) & (df_2[column] == df_3[column]))
        agreement_3_list = list(agreement_3_list[agreement_3_list].index)
        dffinal.loc[agreement_3_list, column] = df1orig.loc[agreement_3_list, column]

        agreement_1_2_list = ((df_1[column] == df_2[column]))
        agreement_1_2_list = list(agreement_1_2_list[agreement_1_2_list].index)
        dffinal.loc[agreement_1_2_list, column] = df1orig.loc[agreement_1_2_list, column]

        agreement_1_3_list = ((df_1[column] == df_3[column]))
        agreement_1_3_list = list(agreement_1_3_list[agreement_1_3_list].index)
        dffinal.loc[agreement_1_3_list, column] = df1orig.loc[agreement_1_3_list, column]

        agreement_2_3_list = ((df_2[column] == df_3[column]))
        agreement_2_3_list = list(agreement_2_3_list[agreement_2_3_list].index)
        dffinal.loc[agreement_2_3_list, column] = df2orig.loc[agreement_2_3_list, column]
                
    return dffinal

In [4]:
NMJ_resolved = conflict_resolver(df_NMJ_1, df_NMJ_2, df_NMJ_3, 'NMJ', ['General comments', 'Is figure quality too poor?', 'Flag this gene as a potential one to use for a figure in the paper?', 'What patterns does the RNA make?', 'What patterns does the protein make?', 'Is RNA distribution cell-type specific?', 'Is protein distribution cell-type specific?'])

/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [5]:
CB_resolved = conflict_resolver(df_CB_1, df_CB_2, df_CB_3, 'CB', ['General comments', 'Is figure quality too poor?', 'Flag this gene as a potential one to use for a figure in the paper?', 'If there is RNA expression, what patterns does the RNA make?', 'If there is protein expression, what patterns does the protein make?'])

/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [6]:
MB_resolved = conflict_resolver(df_MB_1, df_MB_2, df_MB_3, 'MB', ['General comments', 'Is figure quality too poor?', 'Flag this gene as a potential one to use for a figure in the paper?', 'If there is RNA expression, what patterns does the RNA make?', 'If there is protein expression, what patterns does the protein make?'])

/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [7]:
OL_resolved = conflict_resolver(df_OL_1, df_OL_2, df_OL_3, 'OL', ['General comments', 'Flag this gene as a potential one to use for a figure in the paper?', 'If there is RNA expression, what patterns does the RNA make?', 'If there is protein expression, what patterns does the protein make?'])

/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [8]:
VNC_resolved = conflict_resolver(df_VNC_1, df_VNC_2, df_VNC_3, 'VNC', ['General comments', 'Flag this gene as a potential one to use for a figure in the paper?', 'If there is RNA expression, what patterns does the RNA make?', 'If there is protein expression, what patterns does the protein make?', 'Is RNA present in midline glia?', 'Is protein present in midline glia?'])

/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [9]:
nerve_resolved = conflict_resolver(df_nerve_1, df_nerve_2, df_nerve_3, 'nerve_glia', ['General comments', 'Flag this gene as a potential one to use for a figure in the paper?', 'What patterns does the RNA make?', 'What patterns does the protein make?'])


/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)
/Users/mariakiourlappou/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexes/base.py:6982: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


# Table assessing severity of conflicts

In [11]:
# Input are the resolved dataframes

def just_conflicts(df, df_1, df_2, df_3):
    mask = pd.isnull(df).stack()
    list_conflicts = mask.loc[mask].index.tolist()
    
    nan_original_values = []
    for df in [df_1, df_2, df_3]:
        for pair in list_conflicts:
            nan_original_values.append(df.set_index('figure_id').loc[pair])
    
    shape = int(len(nan_original_values)/3)
    index = pd.MultiIndex.from_tuples(list_conflicts, names=["Figure_ID", "Question"])
    s = pd.Series(np.reshape(nan_original_values, (3, shape))[0], index=index, name= 'Scorer 1')
    
    conflicts_each = pd.DataFrame(s)
    conflicts_each['Scorer 2'] = np.reshape(nan_original_values, (3, shape))[1]
    conflicts_each['Scorer 3'] = np.reshape(nan_original_values, (3, shape))[2]
    
    return conflicts_each

In [12]:
CB_conflicts = just_conflicts(CB_resolved, df_CB_1, df_CB_2, df_CB_3)

In [13]:
NMJ_conflicts = just_conflicts(NMJ_resolved, df_NMJ_1, df_NMJ_2, df_NMJ_3)

In [14]:
MB_conflicts = just_conflicts(MB_resolved, df_MB_1, df_MB_2, df_MB_3)

In [15]:
OL_conflicts = just_conflicts(OL_resolved, df_OL_1, df_OL_2, df_OL_3)

In [16]:
VNC_conflicts = just_conflicts(VNC_resolved, df_VNC_1, df_VNC_2, df_VNC_3)

In [17]:
nerve_conflicts = just_conflicts(nerve_resolved, df_nerve_1, df_nerve_2, df_nerve_3)

## Resolving conflicts by assigning mean value to level questions

In [18]:
def conflict_resolver_level(df_conflicts, df):
    df_conflicts.replace({'None': 1, 'Low': 2, 'Moderate': 3, 'High': 4}, inplace=True)
    df_conflicts = df_conflicts[~df_conflicts['Scorer 1'].str.contains('Yes|No|nan', na=False)]
    df_conflicts['mean'] = df_conflicts.mean(axis=1)
    index_list_low = df_conflicts[(df_conflicts['mean'] == 2.0)].index.values
    index_list_moderate = df_conflicts[(df_conflicts['mean'] == 3.0)].index.values
    
    for pair in index_list_low:
        df.loc[pair] = 'Low'
        
    for pair in index_list_moderate:
        df.loc[pair] = 'Moderate'

    return df

In [19]:
MB_level_resolved = conflict_resolver_level(MB_conflicts, MB_resolved)

<ipython-input-18-d9317ecc5f43>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflicts['mean'] = df_conflicts.mean(axis=1)


In [20]:
CB_level_resolved = conflict_resolver_level(CB_conflicts, CB_resolved)

<ipython-input-18-d9317ecc5f43>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflicts['mean'] = df_conflicts.mean(axis=1)


In [21]:
NMJ_level_resolved = conflict_resolver_level(NMJ_conflicts, NMJ_resolved)

<ipython-input-18-d9317ecc5f43>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflicts['mean'] = df_conflicts.mean(axis=1)


In [22]:
OL_level_resolved = conflict_resolver_level(OL_conflicts, OL_resolved)

<ipython-input-18-d9317ecc5f43>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflicts['mean'] = df_conflicts.mean(axis=1)


In [23]:
VNC_level_resolved = conflict_resolver_level(VNC_conflicts, VNC_resolved)

<ipython-input-18-d9317ecc5f43>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflicts['mean'] = df_conflicts.mean(axis=1)


In [24]:
nerve_level_resolved = conflict_resolver_level(nerve_conflicts, nerve_resolved)

<ipython-input-18-d9317ecc5f43>:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_conflicts['mean'] = df_conflicts.mean(axis=1)
<ipython-input-18-d9317ecc5f43>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conflicts['mean'] = df_conflicts.mean(axis=1)


### Creating the file to import the cases to Zegami

In [32]:
just_conflicts(MB_level_resolved, df_MB_1, df_MB_2, df_MB_3).to_csv('scoringResults/MB/MB_conflicts.csv')

In [33]:
just_conflicts(CB_level_resolved, df_CB_1, df_CB_2, df_CB_3).to_csv('scoringResults/CB/CB_conflicts.csv')

In [34]:
just_conflicts(NMJ_level_resolved, df_NMJ_1, df_NMJ_2, df_NMJ_3).to_csv('scoringResults/NMJ/NMJ_conflicts.csv')

In [35]:
just_conflicts(OL_level_resolved, df_OL_1, df_OL_2, df_OL_3).to_csv('scoringResults/OL/OL_conflicts.csv')

In [36]:
just_conflicts(VNC_level_resolved, df_VNC_1, df_VNC_2, df_VNC_3).to_csv('scoringResults/VNC/VNC_conflicts.csv')

In [37]:
just_conflicts(nerve_level_resolved, df_nerve_1, df_nerve_2, df_nerve_3).to_csv('scoringResults/nerve/nerve_conflicts.csv')

### Importing the resolved cases from Zegami

In [100]:
resolved_CB_zegami = pd.read_csv('scoringResults/CB/CB_conflicts_toberesolved.csv')
resolved_MB_zegami = pd.read_csv('scoringResults/MB/MB_conflicts_toberesolved.csv')
resolved_OL_zegami = pd.read_csv('scoringResults/OL/OL_conflicts_toberesolved.csv')
resolved_NMJ_zegami = pd.read_csv('scoringResults/NMJ/NMJ_conflicts_toberesolved.csv')
resolved_VNC_zegami = pd.read_csv('scoringResults/VNC/VNC_conflicts_toberesolved.csv')
resolved_nerve_zegami = pd.read_csv('scoringResults/nerve/nerve_conflicts_toberesolved.csv')

In [101]:
for i in range(0, len(resolved_OL_zegami)):
    OL_level_resolved.loc[resolved_OL_zegami.loc[i, 'Figure_ID'], resolved_OL_zegami.loc[i, 'Question']] \
    = resolved_OL_zegami.loc[i, 'Tags']
    
for i in range(0, len(resolved_CB_zegami)):
    CB_level_resolved.loc[resolved_CB_zegami.loc[i, 'Figure_ID'], resolved_CB_zegami.loc[i, 'Question']] \
    = resolved_CB_zegami.loc[i, 'Tags']

for i in range(0, len(resolved_MB_zegami)):
    MB_level_resolved.loc[resolved_MB_zegami.loc[i, 'Figure_ID'], resolved_MB_zegami.loc[i, 'Question']] \
    = resolved_MB_zegami.loc[i, 'Tags']
    
for i in range(0, len(resolved_NMJ_zegami)):
    NMJ_level_resolved.loc[resolved_NMJ_zegami.loc[i, 'Figure_ID'], resolved_NMJ_zegami.loc[i, 'Question']] \
    = resolved_NMJ_zegami.loc[i, 'Tags']

for i in range(0, len(resolved_VNC_zegami)):
    VNC_level_resolved.loc[resolved_VNC_zegami.loc[i, 'Figure_ID'], resolved_VNC_zegami.loc[i, 'Question']] \
    = resolved_VNC_zegami.loc[i, 'Tags']

for i in range(0, len(resolved_nerve_zegami)):
    nerve_level_resolved.loc[resolved_nerve_zegami.loc[i, 'Figure_ID'], resolved_nerve_zegami.loc[i, 'Question']] \
    = resolved_nerve_zegami.loc[i, 'Tags']

### Creating the FINAL RESOLVED annotations sheet

In [102]:
#OL_level_resolved.to_csv('OL_resolved.csv')
#MB_level_resolved.to_csv('MB_resolved.csv')
#CB_level_resolved.to_csv('CB_resolved.csv')
#NMJ_level_resolved.to_csv('NMJ_resolved.csv')
#VNC_level_resolved.to_csv('VNC_resolved.csv')
#nerve_level_resolved.to_csv('nerve_resolved.csv')